In [1]:
from bs4 import BeautifulSoup
import requests,time,csv
import urllib.parse
from urllib.parse import urlparse, parse_qs
from csv import writer

In [2]:
def parsed_url(substr):
    parts = substr.split("&&&")
    category, location, district = parts[0], parts[1], parts[2]
    return category, location, district

def parsed_number(num):  
    faxx,phonee=[],[]
    for data in num:
        if 'Phone' in data:
            extra = data.split(':')
            phonee.append(extra[1])
        elif 'Mobile' in data:
            extra=data.split(':')
            phonee.append(extra[1])
        elif 'Fax' in data:
            extra=data.split(':')
            faxx.append(extra[1])
        else:
            phonee.append(num[0])
            try: 
                faxx.append(num[1])
            except:
                pass
            break
    return phonee,faxx


with open('------------------.csv','a', encoding= 'utf8' ,newline='') as f:
    thewriter = writer(f)
#     header= ["Category","City","District","Link","Name", "Address","Phone","Fax","Website","Email","Description"]
#     thewriter.writerow(header)
    with open('----------------------', 'r') as file:
        reader = csv.reader(file)
        for count, row in enumerate(reader):
            for string in row:
                category, location, district ='','',''
                substrings = string.split('joining')
                url = substrings[0]
                category, location, district =parsed_url(substrings[1])
                print(url,count)
                sa_key = '475004445dc448db8bc870f4b1f61863'
                sa_api = 'https://api.scrapingant.com/v2/general'
                qParams = {'url': url, 'x-api-key': sa_key}
                reqUrl = f'{sa_api}?{urllib.parse.urlencode(qParams)}'
                r = requests.get(reqUrl)
                soup = BeautifulSoup(r.content, 'html.parser')
                name,add,web,email= '','','',''
                phone,des,number= [],[],[]
                try:
                    name = soup.find(id='cntct-name').text
                except:
                    pass
                try:
                    add = soup.find(id='cp-street').text
                except:
                    pass
                try:
                    webb = soup.find(id='cp-website').text
                    if 'Website' in webb:
                        words = webb.split()
                        web = words[1]
                    else:
                        web = webb
                except:
                    pass
                try: 
                    phone_no = soup.find(id='cp-mainPhone-1').text
                    number.append(phone_no)
                except:
                    pass
                try:
                    fax = soup.find(id='cp-mainPhone-2').text
                    number.append(fax)
                except:
                    pass
                phone,fax = parsed_number(number)

                try:
                    div_email =soup.find('div', class_='col-sm-10 col-9 contact-data-container').text
                    if '@' in div_email:
                        email = div_email
                except:
                    pass
                try: 
                    desc = soup.find(id='ulkw')
                    li = desc.find_all('li')
                    for i in li:
                        des.append(i.text)
                except:
                    pass
                info = [category, location, district,url,name,add,phone,fax,web,email,des]
                print(info)
                thewriter.writerow(info)

